# Setup - Install Libraries

In [1]:
# Run the following commands once, in order to install libraries - DO NOT Uncomment this line.

# Uncomment below lines
#
# !pip3 install --upgrade pip
# !pip install google-cloud-bigquery
# !pip install pandas-gbq -U
# !pip install db-dtypes
# !pip install packaging --upgrade

   ---------------------------------------- 0.0/28.0 MB ? eta -:--:--
   - -------------------------------------- 1.0/28.0 MB 5.6 MB/s eta 0:00:05
   --- ------------------------------------ 2.4/28.0 MB 5.6 MB/s eta 0:00:05
   ---- ----------------------------------- 3.4/28.0 MB 5.8 MB/s eta 0:00:05
   ------ --------------------------------- 4.7/28.0 MB 5.8 MB/s eta 0:00:04
   -------- ------------------------------- 6.0/28.0 MB 6.0 MB/s eta 0:00:04
   ---------- ----------------------------- 7.3/28.0 MB 6.0 MB/s eta 0:00:04
   ------------ --------------------------- 8.7/28.0 MB 6.0 MB/s eta 0:00:04
   ------------- -------------------------- 9.7/28.0 MB 6.0 MB/s eta 0:00:04
   --------------- ------------------------ 11.0/28.0 MB 5.9 MB/s eta 0:00:03
   ----------------- ---------------------- 12.1/28.0 MB 5.9 MB/s eta 0:00:03
   ------------------- -------------------- 13.6/28.0 MB 6.0 MB/s eta 0:00:03
   --------------------- ------------------ 14.9/28.0 MB 6.0 MB/s eta 0:00:03
  

# Import libraries

In [1]:
# Import libraries
from google.cloud import bigquery
import pandas as pd
from pandas_gbq import to_gbq
import os

print('Libraries imported successfully')

Libraries imported successfully


In [2]:
# Set the environment variable for Google Cloud credentials
# Place the path in which the .json file is located.

# Example (if .json is located in the same directory with the notebook)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "at-arch-416714-6f9900ec7.json"

# -- YOUR CODE GOES BELOW THIS LINE
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/Mouxios Giorgos/Downloads/Data Analytics/warm-influence-480112-t2-3027da7e0b9c.json" # Edit path
# -- YOUR CODE GOES ABOVE THIS LINE

In [3]:
# Set your Google Cloud project ID and BigQuery dataset details

# -- YOUR CODE GOES BELOW THIS

project_id = 'warm-influence-480112-t2' # Edit with your project id
dataset_id = 'staging_db' # Modify the necessary schema name: staging_db, reporting_db etc.
table_id = 'stg_store' # Modify the necessary table name: stg_customer, stg_city etc.

# -- YOUR CODE GOES ABOVE THIS LINE

# SQL Query

In [4]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# -- YOUR CODE GOES BELOW THIS LINE

# Define your SQL query here
query ='''
with base as (
  select *
  from `warm-influence-480112-t2.pagila_production_mouxpublic.store`
),
final as (
  select
    store_id,
    manager_staff_id AS store_manager_staff_id,
    address_id AS store_address_id,
    last_update AS store_last_update
  FROM base
)
select * from final

'''

# -- YOUR CODE GOES ABOVE THIS LINE

# Execute the query and store the result in a dataframe
df = client.query(query).to_dataframe()

# Explore some records
df.head()

C:\Users\Mouxios Giorgos\python312\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,store_id,store_manager_staff_id,store_address_id,store_last_update
0,1,1,1,2022-02-15 09:57:12+00:00
1,2,2,2,2022-02-15 09:57:12+00:00


# Write to BigQuery

In [5]:
# Define the full table ID
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# -- YOUR CODE GOES BELOW THIS LINE
# Define table schema based on the project description

schema = [
    bigquery.SchemaField('store_id', 'INTEGER'),
    bigquery.SchemaField('store_manager_staff_id', 'INTEGER'),
    bigquery.SchemaField('store_address_id', 'INTEGER'),
    bigquery.SchemaField('store_last_update', 'DATETIME')
]


# -- YOUR CODE GOES ABOVE THIS LINE

In [6]:
# Create a BigQuery client
client = bigquery.Client(project=project_id)

# Check if the table exists
def table_exists(client, full_table_id):
    try:
        client.get_table(full_table_id)
        return True
    except Exception:
        return False

# Write the dataframe to the table (overwrite if it exists, create if it doesn't)
if table_exists(client, full_table_id):
    # If the table exists, overwrite it
    destination_table = f"{dataset_id}.{table_id}"
    # Write the dataframe to the table (overwrite if it exists)
    to_gbq(df, destination_table, project_id=project_id, if_exists='replace')
    print(f"Table {full_table_id} exists. Overwritten.")
else:
    # If the table does not exist, create it
    job_config = bigquery.LoadJobConfig(schema=schema)
    job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
    job.result()  # Wait for the job to complete
    print(f"Table {full_table_id} did not exist. Created and data loaded.")

Table warm-influence-480112-t2.staging_db.stg_store did not exist. Created and data loaded.
